In [2]:
#各ライブラリのimport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
#重回帰分析
#読み込みから標準化まで(trainのみ)
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)
#pwrカラム作成
df_train['pwr'] = df_train['weight']/df_train['horsepower']
#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#エンコーディング
df_number = pd.get_dummies(df_train , columns = ['origin' , 'manufacturers name'] , dtype=int)

#標準化
#目的変数の標準化は必要ない
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_number.drop(['id' , 'car name', 'mpg','cylinders' , 'weight','horsepower'] , axis = 1)
df_train_std = pd.DataFrame(scaler.fit_transform(df_train_drop), columns = df_train_drop.columns)

#特徴量の分割
features = df_train_std
print(features)
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
target = df_train['mpg']
#print(df_train_std.describe())
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.2 , random_state = 0)
#ライブラリのimport
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#学習と評価
model.fit(features_train , target_train)
pred = model.predict(features_test).reshape(-1,1)

#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(target_test, pred)) 

     displacement  acceleration  model year       pwr  origin_1  origin_2  \
0       -0.492207      0.130705    1.537995  0.127028  0.794448 -0.485824   
1       -0.960864     -0.611386    0.747140 -0.497811 -1.258736  2.058359   
2       -0.278255     -0.054818   -0.043716 -0.494216 -1.258736 -0.485824   
3       -0.950675     -0.425863   -0.570953 -0.238987  0.794448 -0.485824   
4       -0.960864      0.613064    1.274377  0.764194 -1.258736 -0.485824   
..            ...           ...         ...       ...       ...       ...   
194     -1.001616      1.318050    1.010758  0.636341 -1.258736 -0.485824   
195      1.209220     -1.056640    0.483521 -0.800857  0.794448 -0.485824   
196      1.229596     -1.353477   -1.625426 -1.393877  0.794448 -0.485824   
197     -0.950675      2.987754    1.010758  4.067996 -1.258736  2.058359   
198     -0.869170     -0.054818   -0.834571 -0.910435 -1.258736  2.058359   

     origin_3  manufacturers name_amc  manufacturers name_audi  \
0   -0.49

471535892278.3837

In [4]:
#重回帰分析
#読み込みから標準化まで(trainのみ)
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)

#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_train_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train['manufacturers name'].values.reshape(-1,1)) , columns = df_train['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_train_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_train_number = pd.merge(df_train_manufacturers_number , df_train_origin_number , how = 'outer' , right_index = True , left_index=True)
df_train = pd.merge(df_train , df_train_number , how = 'outer' , right_index = True , left_index=True)
df_train = df_train.dropna(how = 'any')

#標準化
#目的変数の標準化は必要ない
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'manufacturers name' , 'car name', 'origin', 'mpg'] , axis = 1)
df_train_std = pd.DataFrame(scaler.fit_transform(df_train_drop), columns = df_train_drop.columns)

#特徴量の分割

features = df_train_std.drop(['cylinders' , 'weight'] , axis = 1)
target = df_train['mpg']
#print(df_train_std.describe())
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.25 , random_state = 0)
#ライブラリのimport
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#学習と評価
model.fit(features_train , target_train)
pred = model.predict(features_test).reshape(-1,1)

#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(target_test, pred)) 

c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


16208455453677.56

In [9]:
#get_dummiesでエンコードver
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
df_train_test.isnull().sum()

df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train_test.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train_test.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train_test.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train_test.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_train_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_train_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#pwrカラム作成
df_train_test['pwr'] = df_train_test['weight']/df_train_test['horsepower']
#メーカーカラム作成
df_split = df_train_test['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#one-hot
#get_dummies(扱うdf,columns=カテゴリ変数を含むカラム,dtype=データ型)
#元のdfに数値化したカラムを結合してくれるから便利
df_number = pd.get_dummies(df_train_test , columns = ['origin' , 'manufacturers name'] , dtype=int)
#:199だと198番目までが選択される
df_train = df_number.iloc[:199 , :]
df_test = df_number.iloc[199: , :]

#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'car name'  , 'mpg','weight' ,'horsepower','cylinders'] , axis = 1)
df_test_drop = df_test.drop(['id' , 'car name' , 'mpg','weight' ,'horsepower','cylinders'] , axis = 1)

scaler.fit(df_train_drop)
df_train_std= pd.DataFrame(scaler.transform(df_train_drop), columns = df_train_drop.columns)
df_test_std = pd.DataFrame(scaler.transform(df_test_drop), columns = df_test_drop.columns)
print(df_train_std)

#目的変数の分離
features_train = df_train_std
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
target_train = df_train['mpg']
print(df_train['mpg'])
features_test = df_test_std

#学習
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(features_train , target_train)
#予測
df_pred = pd.DataFrame(model.predict(features_test))
print(df_pred[df_pred <= 0])
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')

df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)

#評価
#multiple regression analysisの頭文字をとってMRA_sample_submit.csv
df_pred.to_csv('MRA_sample_submit.csv' , header = False , index = False)

     displacement  acceleration  model year       pwr  origin_1  origin_2  \
0       -0.492207      0.130705    1.537995  0.119333  0.794448 -0.485824   
1       -0.960864     -0.611386    0.747140 -0.494432 -1.258736  2.058359   
2       -0.278255     -0.054818   -0.043716 -0.490901 -1.258736 -0.485824   
3       -0.950675     -0.425863   -0.570953 -0.240195  0.794448 -0.485824   
4       -0.960864      0.613064    1.274377  0.745209 -1.258736 -0.485824   
..            ...           ...         ...       ...       ...       ...   
194     -1.001616      1.318050    1.010758  0.619621 -1.258736 -0.485824   
195      1.209220     -1.056640    0.483521 -0.792109  0.794448 -0.485824   
196      1.229596     -1.353477   -1.625426 -1.374619  0.794448 -0.485824   
197     -0.950675      2.987754    1.010758  3.990464 -1.258736  2.058359   
198     -0.869170     -0.054818   -0.834571 -0.899744 -1.258736  2.058359   

     origin_3  manufacturers name_amc  manufacturers name_audi  \
0   -0.49

In [ ]:
#trainのみで学習評価
print(df_train.describe())
#特徴量の分割
features = df_train.drop(['mpg','cylinders' , 'weight'] , axis = 1)
target = df_train['mpg']
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.25 , random_state = 0)
#ライブラリのimport
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#学習と評価
model.fit(features_train , target_train)
pred = model.predict(features_test)
#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(target_test, pred)) 

                mpg   cylinders  displacement    horsepower        weight  \
count  1.990000e+02  199.000000  1.990000e+02  1.990000e+02  1.990000e+02   
mean  -2.588661e-16    0.000000 -4.016887e-17  1.517491e-16 -1.338962e-16   
std    1.002522e+00    1.002522  1.002522e+00  1.002522e+00  1.002522e+00   
min   -1.967979e+00   -1.399932 -1.144251e+00 -1.553318e+00 -1.554154e+00   
25%   -8.108763e-01   -0.790333 -8.691699e-01 -7.457911e-01 -8.148899e-01   
50%   -3.947458e-02   -0.790333 -4.412662e-01 -3.066095e-01 -2.223776e-01   
75%    7.962106e-01    0.428863  6.794342e-01  3.309121e-01  6.636392e-01   
max    2.609005e+00    1.648060  2.757824e+00  3.376849e+00  2.759139e+00   

       acceleration    model year       dodge          vw      toyota  ...  \
count  1.990000e+02  1.990000e+02  199.000000  199.000000  199.000000  ...   
mean  -2.454764e-16  1.687093e-15    0.080402    0.010050    0.010050  ...   
std    1.002522e+00  1.002522e+00    0.272600    0.099997    0.099997  .

0.4146033228104701